# VacationPy
---

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,9.81,46,20,8.23,AR,1696006230
1,1,amga,60.8953,131.9608,-2.50,72,23,1.69,RU,1696006603
2,2,tidjikja,18.5564,-11.4271,41.52,7,80,5.31,MR,1696006271
3,3,ilulissat,69.2167,-51.1000,2.01,64,75,1.54,GL,1696006232
4,4,tromso,69.6496,18.9570,10.01,93,75,0.00,NO,1696006274


In [8]:
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 600,
    tiles ="OSM",
    size = "Humidity",
    color = "City"
)

humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# We want cities that have our ideal weather condition: a max temperature lower than 27 degrees but higher than 21, wind speed less than 4.5 m/s, and zero cloudiness
ideal_condition_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_condition_df.dropna()

# Display sample data
ideal_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,nagykoros,47.0342,19.7786,22.97,53,0,0.00,HU,1696006603
42,42,saint-pierre,-21.3393,55.4781,22.82,73,0,1.03,RE,1696006605
108,108,mokopane,-24.1944,29.0097,24.04,28,0,2.48,ZA,1696006608
129,129,calama,-22.4667,-68.9333,25.27,7,0,2.57,CL,1696006610
130,130,shaowu,27.3409,117.4831,23.65,93,0,1.11,CN,1696006610
132,132,maniago,46.1706,12.7078,25.47,44,0,1.03,IT,1696006610
140,140,haiku-pauwela,20.9219,-156.3051,26.13,78,0,1.54,US,1696006260
157,157,kailua-kona,19.6406,-155.9956,24.77,72,0,3.09,US,1696006268
197,197,port saint john's,-31.6229,29.5448,22.09,77,0,4.20,ZA,1696006255
261,261,thabazimbi,-24.5917,27.4116,25.11,26,0,1.61,ZA,1696006617


In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
import numpy as np 

hotel_df = ideal_condition_df.copy(deep=True)
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,nagykoros,HU,47.0342,19.7786,53,None
42,saint-pierre,RE,-21.3393,55.4781,73,None
108,mokopane,ZA,-24.1944,29.0097,28,None
129,calama,CL,-22.4667,-68.9333,7,None
130,shaowu,CN,27.3409,117.4831,93,None
132,maniago,IT,46.1706,12.7078,44,None
140,haiku-pauwela,US,20.9219,-156.3051,78,None
157,kailua-kona,US,19.6406,-155.9956,72,None
197,port saint john's,ZA,-31.6229,29.5448,77,None
261,thabazimbi,ZA,-24.5917,27.4116,26,None


In [28]:
# Set parameters to search for a hotel
from pprint import pprint 

radius = 10000

params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "filter": None,
    "bias": None,
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Set up the three elements of the params dictionary
    latitude = row['Lat']
    longitude = row['Lng']
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Get the response from the API
    response = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nagykoros - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
mokopane - nearest hotel: Oasis Hotel
calama - nearest hotel: Apart Hotel Geotel Calama
shaowu - nearest hotel: Jinqiu Hotel
maniago - nearest hotel: Montenegro
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kailua-kona - nearest hotel: Kona Seaside Hotel
port saint john's - nearest hotel: Outback Inn
thabazimbi - nearest hotel: Maroela Guest Lodge
east london - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
okayama - nearest hotel: ザ・ワンファイぷ岡山
qom - nearest hotel: لاله
waterloo - nearest hotel: Delta Waterloo


,City,Country,Lat,Lng,Humidity,Hotel Name
5,nagykoros,HU,47.0342,19.7786,53,No hotel found
42,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
108,mokopane,ZA,-24.1944,29.0097,28,Oasis Hotel
129,calama,CL,-22.4667,-68.9333,7,Apart Hotel Geotel Calama
130,shaowu,CN,27.3409,117.4831,93,Jinqiu Hotel
132,maniago,IT,46.1706,12.7078,44,Montenegro
140,haiku-pauwela,US,20.9219,-156.3051,78,Inn At Mama's Fish House
157,kailua-kona,US,19.6406,-155.9956,72,Kona Seaside Hotel
197,port saint john's,ZA,-31.6229,29.5448,77,Outback Inn
261,thabazimbi,ZA,-24.5917,27.4116,26,Maroela Guest Lodge


In [44]:
# Configure the map
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color="Hotel Name"
)

# Display the map plot
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)